# Judge, Jury and classifier

## Introduction

In 2002, Andrew Martin, a professor of political science at Washington University in St. Louis, decided to instead predict decisions using a statistical model built from data Together with his colleagues, he decided to test this model against a panel of experts

Martin used a method called Decission Trees

Why not logistic regression?

* Logistic regression models are generally not interpretable
* Model coefficients indicate importance and relative effect of variables, but do not give a simple explanation of how decision is made

## About the data

Cases from 1994 through 2001.

In this period, the same nine justices presided SCOTUS:

* Breyer, Ginsburg, Kennedy, O’Connor, Rehnquist (Chief Justice), Scalia, Souter, Stevens, Thomas.
* Rare data set - longest period of time with the same set of justices in over 180 years.

We will focus on predicting **Justice Stevens’ decisions**:

* Started out moderate, but became more liberal
* Self-proclaimmed conservative

##### Libraries

In [1]:
# standard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# style like ggplot
sns.set()

## The variables

In this problem, our **dependent variable** is whether or not Justice Stevens voted to reverse the lower court decision. This is a binary variable, **Reverse**, taking values:

* 1 if Justice Stevens decided to **reverse** or overturn the lower court decision.
* 0 if Justice Stevens voted to **affirm** or maintain the lower court decision.

Our **independent variables** are six different properties of the case.

* **Circuit**: Circuit court of origin, 13 courts (1st - 11th , DC, FED).
* **Issue**: Issue area of case, 11 areas (e.g., civil rights, federal taxation).
* **Petitioner**: type of petitioner, 12 categories (e.g., US, an employer).
* **Respondent**: type of respondent 12 categories (same as for Petitioner).
* **LowerCourt**: Ideological direction of lower court decision (this was based on the judgment by the authors of the study), 2 categories:
    * **conservative**
    * **liberal**
* **Unconst**: whether petitioner argued that a law/practice was unconstitutional, binary variable.

## Loading the data

In [2]:
# The data
data = pd.read_csv('../../../datasets/Decision_trees/stevens.csv')
data.head()

,Docket,Term,Circuit,Issue,Petitioner,Respondent,LowerCourt,Unconst,Reverse
0,93-1408,1994,2nd,EconomicActivity,BUSINESS,BUSINESS,liberal,0,1
1,93-1577,1994,9th,EconomicActivity,BUSINESS,BUSINESS,liberal,0,1
2,93-1612,1994,5th,EconomicActivity,BUSINESS,BUSINESS,liberal,0,1
3,94-623,1994,1st,EconomicActivity,BUSINESS,BUSINESS,conser,0,1
4,94-1175,1995,7th,JudicialPower,BUSINESS,BUSINESS,conser,0,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 9 columns):
Docket        566 non-null object
Term          566 non-null int64
Circuit       566 non-null object
Issue         566 non-null object
Petitioner    566 non-null object
Respondent    566 non-null object
LowerCourt    566 non-null object
Unconst       566 non-null int64
Reverse       566 non-null int64
dtypes: int64(3), object(6)
memory usage: 39.9+ KB


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 9 columns):
Docket        566 non-null object
Term          566 non-null int64
Circuit       566 non-null object
Issue         566 non-null object
Petitioner    566 non-null object
Respondent    566 non-null object
LowerCourt    566 non-null object
Unconst       566 non-null int64
Reverse       566 non-null int64
dtypes: int64(3), object(6)
memory usage: 39.9+ KB


Como podemos ver, la mayoría de las variables son del tipo categórico, por lo tanto, es conveniente realizar un árbol de decisiones. Este nos permitirá una mayor interpretación de los resultados que un modelo logístico.

## Data wrangling

Some of the variables are not interesting for our purpose, namely Docket and Term and we remove them.

In [5]:
data = data.drop(columns=['Docket', 'Term'], axis=1)
data.tail()

,Circuit,Issue,Petitioner,Respondent,LowerCourt,Unconst,Reverse
561,6th,CriminalProcedure,OTHER,US,conser,0,0
562,5th,CriminalProcedure,OTHER,US,liberal,0,1
563,9th,CriminalProcedure,OTHER,US,conser,1,0
564,DC,CivilRights,STATE,US,conser,0,0
565,9th,CivilRights,STATE,US,liberal,0,0


## Logistic regression

Para comprobar que estamos en lo cierto en cuanto a la mejor interpretación de un modelo de árbol frente a un modelo logístico, vamos a realizar ambos modelos y comprobar con cual obtenemos mejores resultados.

##### Libraries for logistic regression